In [171]:
# Imports 
import numpy as np 
import cv2 
import dlib
import matplotlib.pyplot as plt
import os
from PIL import Image
import imageio
import subprocess
%matplotlib inline

In [173]:
def get_affine_transform(src_tri, dst_tri):

    # Calculate the transformation matrix using the inverse:
    #   M = inv(src) * dst
    src_inv = np.linalg.pinv(src_tri)

    # Calculate the transformation matrix M
    affine_matrix = np.dot(dst_tri, src_inv)

    return affine_matrix

def reshape_triangle_for_transformation(triangle):
    triangle = np.transpose(triangle)
    row_of_ones = np.ones((1, triangle.shape[1]))  # Creating a row of ones with the same number of columns as the matrix
    triangle = np.vstack([triangle, row_of_ones])
    return triangle

def custom_warp_affine(src, warp_mat, size):
    height, width = size

    # Generate pixel coordinates for the output image
    x, y = np.meshgrid(np.arange(width), np.arange(height))

    # Flatten the coordinates and add a row of ones for homogeneous coordinates
    coordinates = np.vstack((x.flatten(), y.flatten(), np.ones_like(x.flatten())))

    # Find the inverse transformation
    inv_warp_mat = np.linalg.pinv(warp_mat)

    # Apply the inverse transformation to get coordinates in the original image
    transformed_coordinates = np.dot(inv_warp_mat, coordinates)

    # Extract x and y coordinates
    src_x, src_y = transformed_coordinates[:2]

    # Clip coordinates to be within the valid range
    src_x = np.clip(src_x, 0, src.shape[1] - 1)
    src_y = np.clip(src_y, 0, src.shape[0] - 1)

    # Perform bilinear interpolation for each channel using vectorized operations
    src_indices = np.ravel_multi_index((np.clip(src_y, 0, src.shape[0] - 1).astype(int),
                                        np.clip(src_x, 0, src.shape[1] - 1).astype(int)), src.shape[:2])

    result = np.zeros((height, width, src.shape[2]), dtype=src.dtype)

    for c in range(src.shape[2]):
        result[:, :, c] = src.reshape((-1, src.shape[2]))[src_indices, c].reshape((height, width))

    return result

In [174]:
# Apply affine transform calculated using srcTri and dstTri to src and output an image of size.
def apply_affine_transform(src, srcTri, dstTri, size) :
    # Given a pair of triangles, find the affine transform.
    srcTri = reshape_triangle_for_transformation(srcTri)
    dstTri = reshape_triangle_for_transformation(dstTri)
    warpMat = get_affine_transform(srcTri,dstTri)

    # Apply the Affine Transform just found to the src image
    dst = custom_warp_affine(src,warpMat,(size[1],size[0]))

    return dst

# # Warps and alpha blends triangular regions from img1 and img2 to img
def triangle_morphing(img1, img2, result_img, src_triangle, dest_triangle, int_triangle, w):
    # Find bounding rectangle for each triangle
    src_img_bounding_rect = cv2.boundingRect(np.float32([src_triangle]))
    dest_img_bounding_rect = cv2.boundingRect(np.float32([dest_triangle]))
    inter_img_bouding_rect = cv2.boundingRect(np.float32([int_triangle]))
    
    # Zero centers the triangle vertices 
    int_rect = []
    src_rect = []
    dest_rect = []

    for i in range(0, 3):
        int_rect.append(((int_triangle[i][0] - inter_img_bouding_rect[0]), (int_triangle[i][1] - inter_img_bouding_rect[1])))
        src_rect.append(((src_triangle[i][0] - src_img_bounding_rect[0]), (src_triangle[i][1] - src_img_bounding_rect[1])))
        dest_rect.append(((dest_triangle[i][0] - dest_img_bounding_rect[0]), (dest_triangle[i][1] - dest_img_bounding_rect[1])))

    # Get mask by filling the triangle
    mask = np.zeros((inter_img_bouding_rect[3], inter_img_bouding_rect[2], 3), dtype=np.float32)
    cv2.fillConvexPoly(mask, np.int32(int_rect), (1.0, 1.0, 1.0), 16, 0)

    # Apply warpImage to small rectangular patches
    img1_rectangular = img1[src_img_bounding_rect[1]:src_img_bounding_rect[1] + src_img_bounding_rect[3], src_img_bounding_rect[0]:src_img_bounding_rect[0] + src_img_bounding_rect[2]]
    img2_rectangular = img2[dest_img_bounding_rect[1]:dest_img_bounding_rect[1] + dest_img_bounding_rect[3], dest_img_bounding_rect[0]:dest_img_bounding_rect[0] + dest_img_bounding_rect[2]]

    # Apply the transformations to get the warped rectangular patches
    size = (inter_img_bouding_rect[2], inter_img_bouding_rect[3])
    warp_image1 = apply_affine_transform(img1_rectangular, src_rect, int_rect, size)
    warp_image2 = apply_affine_transform(img2_rectangular, dest_rect, int_rect, size)

    # Alpha blend rectangular patches
    blended_warped_img = (1.0 - w) * warp_image1 + w * warp_image2

    # Copy triangular region of the rectangular patch to the output image
    result_img[inter_img_bouding_rect[1]:inter_img_bouding_rect[1]+inter_img_bouding_rect[3], inter_img_bouding_rect[0]:inter_img_bouding_rect[0]+inter_img_bouding_rect[2]] = result_img[inter_img_bouding_rect[1]:inter_img_bouding_rect[1]+inter_img_bouding_rect[3], inter_img_bouding_rect[0]:inter_img_bouding_rect[0]+inter_img_bouding_rect[2]] * (1 - mask) + blended_warped_img * mask


In [175]:
def images_to_video(images, output_file):
    with imageio.get_writer(output_file, mode='I', fps=30) as writer:
            for k in range(len(images)):
                writer.append_data(np.asarray(images[k]))


In [176]:
def face_morph_generation(duration,frame_rate,img1,img2,src_img_points,dest_img_points,lst_of_triangles,output):

    num_images = int(duration*frame_rate)
    frames_for_video = [] # Stores Morphed Frames
  
    for j in range(0, num_images):

        # Convert Mat to float data type
        img1 = np.float32(img1)
        img2 = np.float32(img2)

        # Read array of corresponding points
        points = []
        w = j/(num_images-1)
        
        # Compute weighted average point coordinates
        for i in range(0, len(src_img_points)):
            x = (1 - w) * src_img_points[i][0] + w * dest_img_points[i][0]
            y = (1 - w) * src_img_points[i][1] + w * dest_img_points[i][1]
            points.append((x,y))
        # Allocate space for final output
        morphed_frame = np.zeros(img1.shape, dtype = img1.dtype)

        for i in range(len(lst_of_triangles)):    
            x = int(lst_of_triangles[i][0])
            y = int(lst_of_triangles[i][1])
            z = int(lst_of_triangles[i][2])
            
            src_tri_vertices = [src_img_points[x], src_img_points[y], src_img_points[z]]
            dest_tri_vertices = [dest_img_points[x], dest_img_points[y], dest_img_points[z]]
            morph_tri_vertices = [points[x], points[y], points[z]]

            # Morph one triangle at a time.
            triangle_morphing(img1, img2, morphed_frame, src_tri_vertices, dest_tri_vertices, morph_tri_vertices, w)

        result = Image.fromarray(cv2.cvtColor(np.uint8(morphed_frame), cv2.COLOR_BGR2RGB))
        frames_for_video.append(result)

    # Convert frames to video
    images_to_video(frames_for_video, output)

In [177]:
def read_file_np_array(src_file_name, triangle_list_file_name, dest_file_name):
    return np.genfromtxt(src_file_name, delimiter=",", dtype=np.int64), np.genfromtxt(triangle_list_file_name, delimiter=",", dtype=np.int64), np.genfromtxt(dest_file_name, delimiter=",", dtype=np.int64)


In [178]:
# Define Wrapper Function
def doMorphing(img1, img2, duration, frame_rate, output):
    src_pts, triangles, dest_points = read_file_np_array("src_points.txt","triangles.txt","dest_points.txt")
    face_morph_generation(duration, frame_rate, img1, img2, src_pts, dest_points, triangles,output)
	
current_directory = os.getcwd()
images_folder = os.path.join(current_directory, 'Images')
photo_1 = os.path.join(images_folder, 'hillary_resize.jpg')
photo_2 = os.path.join(images_folder,'ted_resize.jpg')
images = [cv2.imread(photo_1),cv2.imread(photo_2)]
output_path = os.path.join(current_directory, 'Outputs', 'output.mp4')
doMorphing(images[0],images[1],5,10,output_path)